<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [65]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

In [66]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [67]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [68]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [69]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [70]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [71]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [72]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-03 16:02:04
-------------------
qualified stocks: 87
with latest results: 26
still star stocks: 16
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  1.05 L
Current:  1.45 C
-------------------
Today PnL: 67.14 K (0.46%)
Current PnL: -19.16 L (-12.56%)
CY Booked + Current PnL: -6.51 L (-4.27%)
-------------------
Total profit:  2.38 L
Total loss:  -21.55 L
-------------------
Total Booked + Current PnL: 20.47 L (16.29%)
Total Booked PnL: 39.63 L (31.55%)
Curr Year Booked PnL: 12.65 L (8.7%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.32 C
Est FTT PnL: 86.21 L (59.29%)
Deployed:  1.26 C
Current:  1.45 C
CAGR/XIRR %: 8.64%


In [73]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
58,RELIANCE,1533.00,-7.28,67.0,X-LC,5.28,231613.0,21247.0,7527.0,-0.11,10.10,3.25,13.68,37.0,2.82,1.60,28.29,XY25,NTT,REFINERIES
78,TTKPRESTIG,770.00,106.40,57.0,M-SC,1.53,88910.0,-11867.0,11958.0,1.33,-11.78,13.45,0.09,245.0,-0.99,0.62,16.81,OX40N,NTT,DURABLES
79,UNIONBANK,163.00,-5.43,67.0,M-LC,10.46,171551.0,30711.0,13776.0,1.49,21.81,8.03,31.59,66.0,2.23,1.19,54.07,XY24,NTT,BANKS
33,ICICIGI,2252.93,-13.61,60.0,X-MC,5.05,148562.0,12429.0,18154.0,0.70,9.13,12.22,22.47,91.0,0.68,1.03,23.29,X40,ATH,INSURANCE
2,ABBOTINDIA,35195.00,-11.87,31.0,X-MC,2.38,87105.0,-3573.0,18484.0,0.14,-3.94,21.22,16.44,101.0,-0.19,0.60,15.44,X40,ATH,PHARMA


In [74]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,MASFIN,398.61,-15.54,63.0,H-SC,8.78,96060.0,-1920.0,23525.0,4.83,-1.96,24.49,22.05,152.0,-0.08,0.67,39.63,XR,ATH,FINANCE
59,REPCOHOME,880.00,-53.06,74.0,H-SC,6.07,270777.0,-15728.0,273052.0,4.62,-5.49,100.84,89.82,134.0,-0.06,1.88,43.26,XY24,NTT,FINANCE
81,VAIBHAVGBL,521.00,82.92,76.0,H-SC,1.73,158625.0,-24150.0,135212.0,4.49,-13.21,85.24,60.77,125.0,-0.18,1.10,42.07,XR,NTT,JEWELLERY
7,ASIANTILES,137.00,7388.89,68.0,L-SC,9.00,82179.0,-11631.0,88252.0,4.11,-12.40,107.39,81.67,269.0,-0.13,0.57,59.14,XR,NTT,CERAMICS
64,SFL,1287.00,41.82,69.0,M-SC,5.14,187295.0,-74923.0,144742.0,3.80,-28.57,77.28,26.63,229.0,-0.52,1.30,16.45,XY24,NTT,MISC


In [75]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.0,-48.21,40.0,X-SC,37.80,51557.0,-13449.0,164379.0,-1.81,-20.69,318.83,232.18,198.0,-0.08,0.36,0.56,XY24,NTT,MISC
25,GREENPANEL,537.0,206.20,40.0,M-SC,7.17,136591.0,-46487.0,125459.0,-1.76,-25.39,91.85,43.14,230.0,-0.37,0.95,28.48,XY24,NTT,MISC
63,SATIN,274.0,-23.78,60.0,H-SC,3.04,235820.0,-40464.0,183680.0,-1.65,-14.65,77.89,51.83,126.0,-0.22,1.63,14.88,XY24,NTT,FINANCE
41,ITC,452.0,-37.50,53.0,X-LC,1.19,202422.0,2284.0,18603.0,-1.52,1.14,9.19,10.44,4.0,0.12,1.40,6.77,X40,NTT,FMCG
69,STARHEALTH,761.0,19.06,55.0,H-SC,13.82,264928.0,-16195.0,149048.0,-1.50,-5.76,56.26,47.26,144.0,-0.11,1.84,41.94,XY24,NTT,INSURANCE


In [76]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [77]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.0,106.40,57.0,M-SC,1.53,88910.0,-11867.0,11958.0,1.33,-11.78,13.45,0.09,245.0,-0.99,0.62,16.81,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.42,50.0,H-SC,2.73,224235.0,-45432.0,81756.0,0.48,-16.85,36.46,13.47,138.0,-0.56,1.55,14.02,XY24,NTT,PAINTS
17,CERA,9475.0,-20.76,60.0,H-SC,2.10,144509.0,-31394.0,73411.0,0.91,-17.85,50.80,23.89,149.0,-0.43,1.00,24.80,OX40N,NTT,CERAMICS
73,TATAELXSI,9161.0,-23.71,47.0,H-MC,6.09,103246.0,-28589.0,70816.0,-0.38,-21.69,68.59,32.03,98.0,-0.40,0.72,15.64,OX40N,NTT,IT
67,SIS,528.0,2004.66,41.0,H-SC,3.28,84940.0,-26092.0,49698.0,-0.06,-23.50,58.51,21.26,156.0,-0.53,0.59,14.76,OX40N,NTT,MISC


In [78]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,UNIONBANK,163.0,-5.43,67.0,M-LC,10.46,171551.0,30711.0,13776.0,1.49,21.81,8.03,31.59,66.0,2.23,1.19,54.07,XY24,NTT,BANKS


In [79]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAIL,228.00,50.76,61.0,M-MC,9.53,237584.0,12622.0,155024.0,0.82,5.61,65.25,74.53,192.0,0.08,1.65,39.67,XY24,BTT,STEEL
56,RECLTD,446.00,47.87,56.0,M-LC,5.18,208560.0,5390.0,36748.0,1.16,2.65,17.62,20.74,55.0,0.15,1.44,9.70,XY25,NTT,FINANCE
31,HINDZINC,730.22,28.52,50.0,M-LC,10.89,209083.0,4007.0,108556.0,0.87,1.95,51.92,54.89,52.0,0.04,1.45,26.52,X5K,ATH,METALS
46,KPIGREEN,731.05,14.01,76.0,H-SC,5.76,135778.0,10481.0,48446.0,0.65,8.36,35.68,47.03,141.0,0.22,0.94,70.01,MH,ATH,POWER
79,UNIONBANK,163.00,-5.43,67.0,M-LC,10.46,171551.0,30711.0,13776.0,1.49,21.81,8.03,31.59,66.0,2.23,1.19,54.07,XY24,NTT,BANKS


In [80]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
46,KPIGREEN,731.05,14.01,76.0,H-SC,5.76,135778.0,10481.0,48446.0,0.65,8.36,35.68,47.03,141.0,0.22,0.94,70.01,MH,ATH,POWER
85,WHIRLPOOL,2270.00,-38.90,59.0,M-SC,4.25,126804.0,12006.0,82042.0,-1.49,10.46,64.70,81.92,223.0,0.15,0.88,50.12,XR,NTT,DURABLES
36,INDIAMART,4810.62,-53.15,56.0,H-SC,6.33,130740.0,7404.0,124216.0,0.36,6.00,95.01,106.72,119.0,0.06,0.91,30.78,AR,ATH,MISC
31,HINDZINC,730.22,28.52,50.0,M-LC,10.89,209083.0,4007.0,108556.0,0.87,1.95,51.92,54.89,52.0,0.04,1.45,26.52,X5K,ATH,METALS
86,WIPRO,420.00,-15.15,43.0,M-LC,5.33,149110.0,-1835.0,111296.0,-0.07,-1.22,74.64,72.51,53.0,-0.02,1.03,4.73,XR,NTT,IT


In [81]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TATAMOTORS,1065.00,-53.17,19.0,X-LC,24.25,148830.0,-127693.0,237771.0,-0.59,-46.18,159.76,39.81,54.0,-0.54,1.03,4.90,XY24,NTT,AUTO
12,BATAINDIA,2096.00,-40.58,28.0,X-SC,5.78,88852.0,-39818.0,85120.0,-0.01,-30.95,95.80,35.20,219.0,-0.47,0.62,1.98,X40,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-11.87,31.0,X-MC,2.38,87105.0,-3573.0,18484.0,0.14,-3.94,21.22,16.44,101.0,-0.19,0.60,15.44,X40,ATH,PHARMA
23,DMART,5391.45,-18.46,32.0,X-LC,8.98,207800.0,-2607.0,61779.0,0.06,-1.24,29.73,28.12,38.0,-0.04,1.44,22.09,X40N,ATH,FMCG
44,JSWINFRA,342.00,-21.91,33.0,X-MC,4.96,195716.0,-4631.0,35131.0,0.03,-2.31,17.95,15.23,178.0,-0.13,1.36,29.19,X40N,NTT,REALTY


In [82]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,ITC,452.00,-37.50,53.0,X-LC,1.19,202422.0,2284.0,18603.0,-1.52,1.14,9.19,10.44,4.0,0.12,1.40,6.77,X40,NTT,FMCG
66,SIEMENS,4671.50,-2.61,48.0,H-LC,1.25,156375.0,-29720.0,77202.0,1.01,-15.97,49.37,25.51,15.0,-0.38,1.08,15.24,AR,ATH,ELECTRICAL
52,PGHH,17907.65,-30.55,39.0,X-MC,1.65,200865.0,45.0,67752.0,0.30,0.02,33.73,33.76,80.0,0.00,1.39,5.22,X40,ATH,FMCG
28,HAVELLS,2069.16,-5.75,48.0,X-MC,2.87,314433.0,-13558.0,120082.0,0.23,-4.13,38.19,32.48,92.0,-0.11,2.18,6.82,X40,ATH,ELECTRICAL
80,UNITDSPR,1644.00,-4.95,74.0,X-LC,4.35,248987.0,14843.0,33788.0,1.13,6.34,13.57,20.77,86.0,0.44,1.72,13.70,X40N,NTT,BREWERIES


In [83]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-48.21,40.0,X-SC,37.80,51557.0,-13449.0,164379.0,-1.81,-20.69,318.83,232.18,198.0,-0.08,0.36,0.56,XY24,NTT,MISC
12,BATAINDIA,2096.00,-40.58,28.0,X-SC,5.78,88852.0,-39818.0,85120.0,-0.01,-30.95,95.80,35.20,219.0,-0.47,0.62,1.98,X40,NTT,FOOTWEAR
20,COLPAL,3726.84,-5.64,44.0,X-MC,7.40,220040.0,-43325.0,152642.0,-0.84,-16.45,69.37,41.51,84.0,-0.28,1.52,3.26,XY25,ATH,FMCG
51,PAGEIND,51769.93,-27.90,40.0,X-MC,6.92,81520.0,-1990.0,22019.0,-1.07,-2.38,27.01,23.98,82.0,-0.09,0.56,3.30,X40,ATH,APPARELS
10,BAJAJHFL,181.50,-14.48,46.0,X-MC,6.00,180252.0,-20254.0,117416.0,-0.27,-10.10,65.14,48.45,90.0,-0.17,1.25,3.94,X40N,ATH,FINANCE


In [84]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-48.21,40.0,X-SC,37.80,51557.0,-13449.0,164379.0,-1.81,-20.69,318.83,232.18,198.0,-0.08,0.36,0.56,XY24,NTT,MISC
51,PAGEIND,51769.93,-27.90,40.0,X-MC,6.92,81520.0,-1990.0,22019.0,-1.07,-2.38,27.01,23.98,82.0,-0.09,0.56,3.30,X40,ATH,APPARELS
57,RELAXO,1176.00,-40.07,43.0,X-SC,4.58,81677.0,-63483.0,139414.0,-0.39,-43.73,170.69,52.31,136.0,-0.46,0.57,9.33,X40N,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-11.87,31.0,X-MC,2.38,87105.0,-3573.0,18484.0,0.14,-3.94,21.22,16.44,101.0,-0.19,0.60,15.44,X40,ATH,PHARMA
12,BATAINDIA,2096.00,-40.58,28.0,X-SC,5.78,88852.0,-39818.0,85120.0,-0.01,-30.95,95.80,35.20,219.0,-0.47,0.62,1.98,X40,NTT,FOOTWEAR


In [85]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4389.97,-28.69,49.0,X-LC,10.23,283579.0,-62377.0,129085.0,-1.35,-18.03,45.52,19.28,1.0,-0.48,1.96,4.83,X40,ATH,IT
39,INFY,2275.00,-18.88,53.0,X-LC,5.37,316412.0,3524.0,168173.0,0.22,1.13,53.15,54.87,3.0,0.02,2.19,9.50,X40,BTT,IT
41,ITC,452.00,-37.50,53.0,X-LC,1.19,202422.0,2284.0,18603.0,-1.52,1.14,9.19,10.44,4.0,0.12,1.40,6.77,X40,NTT,FMCG
83,VBL,671.64,-15.83,55.0,X-LC,7.52,302890.0,-12952.0,125608.0,1.09,-4.10,41.47,35.67,5.0,-0.10,2.10,9.19,X40N,ATH,FMCG
1,ABB,7934.00,-39.67,53.0,H-LC,5.33,252288.0,-9331.0,128541.0,0.69,-3.57,50.95,45.57,7.0,-0.07,1.75,7.35,AR,NTT,ELECTRICAL


In [86]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COFFEEDAY,80.00,-45.67,44.0,L-SC,28.39,79219.0,-34330.0,74379.0,-1.46,-30.23,93.89,35.27,268.0,-0.46,0.55,93.89,XR,NTT,HOTELS
7,ASIANTILES,137.00,7388.89,68.0,L-SC,9.00,82179.0,-11631.0,88252.0,4.11,-12.40,107.39,81.67,269.0,-0.13,0.57,59.14,XR,NTT,CERAMICS
49,MASFIN,398.61,-15.54,63.0,H-SC,8.78,96060.0,-1920.0,23525.0,4.83,-1.96,24.49,22.05,152.0,-0.08,0.67,39.63,XR,ATH,FINANCE
85,WHIRLPOOL,2270.00,-38.90,59.0,M-SC,4.25,126804.0,12006.0,82042.0,-1.49,10.46,64.70,81.92,223.0,0.15,0.88,50.12,XR,NTT,DURABLES
36,INDIAMART,4810.62,-53.15,56.0,H-SC,6.33,130740.0,7404.0,124216.0,0.36,6.00,95.01,106.72,119.0,0.06,0.91,30.78,AR,ATH,MISC


In [87]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,VAIBHAVGBL,521.00,82.92,76.0,H-SC,1.73,158625.0,-24150.0,135212.0,4.49,-13.21,85.24,60.77,125.0,-0.18,1.10,42.07,XR,NTT,JEWELLERY
59,REPCOHOME,880.00,-53.06,74.0,H-SC,6.07,270777.0,-15728.0,273052.0,4.62,-5.49,100.84,89.82,134.0,-0.06,1.88,43.26,XY24,NTT,FINANCE
46,KPIGREEN,731.05,14.01,76.0,H-SC,5.76,135778.0,10481.0,48446.0,0.65,8.36,35.68,47.03,141.0,0.22,0.94,70.01,MH,ATH,POWER
85,WHIRLPOOL,2270.00,-38.90,59.0,M-SC,4.25,126804.0,12006.0,82042.0,-1.49,10.46,64.70,81.92,223.0,0.15,0.88,50.12,XR,NTT,DURABLES
5,ANGELONE,3033.00,18.88,67.0,X-SC,5.83,210232.0,19226.0,38472.0,2.86,10.07,18.30,30.21,157.0,0.50,1.46,33.05,X40N,NTT,FINANCE


In [88]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.63
1,25,44.03
2,50,75.44


In [89]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

,CurrAlloc%
Conviction,
SC,44.15
LC,32.95
MC,22.88


In [90]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

,CurrAlloc%
Criteria,
XY24,29.98
X40,16.08
X40N,12.44
XY25,11.91
XR,9.96
AR,7.82
OX40N,6.55
X200,1.78
X5K,1.45


In [91]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

,CurrAlloc%
Conviction,
H-SC,24.46
X-LC,22.23
X-MC,18.57
M-SC,12.69
X-SC,5.55
M-LC,5.11
H-LC,4.61
H-MC,2.35
M-MC,1.65


In [92]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.11,-3.65,37.16
IT,12.42,-17.29,80.20
FINANCE,12.24,-7.04,55.56
MISC,7.82,-15.36,73.55
ELECTRICAL,5.77,-8.49,46.95
PAINTS,5.45,-17.16,34.62
BANKS,4.05,-15.29,82.35
INSURANCE,3.81,-0.75,38.31
AUTO,2.74,-45.15,105.73


In [93]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3246079.0,22
XR,1218786.0,13
AR,1159659.0,9
X40,833896.0,12
X40N,673887.0,9
OX40N,540444.0,9
XY25,456520.0,8
SR,267019.0,2
X5K,108556.0,1


In [94]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3303299.0,24
M-SC,1211437.0,15
X-MC,1192924.0,14
X-LC,1066637.0,13
X-SC,549664.0,6
H-LC,273127.0,3
M-LC,270376.0,4
L-SC,251859.0,3
H-MC,248667.0,2


In [95]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1158514.0      6
           AR         830738.0      5
           XR         758488.0      7
M-SC       XY24       696301.0      6
X-MC       XY24       573493.0      4
X-LC       X40        459178.0      5
           XY24       306741.0      2
X-SC       X40N       300165.0      4
X-MC       X40        289598.0      6
H-SC       SR         267019.0      2
           OX40N      240094.0      3
X-LC       X40N       221175.0      3
H-LC       AR         205743.0      2
H-MC       XY24       177851.0      1
X-MC       XY25       177286.0      2
X-SC       XY24       164379.0      1
L-SC       XR         162631.0      2
M-MC       XY24       155024.0      1
X-MC       X40N       152547.0      2
M-SC       OX40N      140306.0      4
           XR         129193.0      2
           AR         123178.0      2
           XY25       122459.0      1
M-LC       XR         111296.0      1
           X5K        108556.0      1
L-SC       OX40N       89228.0      1
X-SC       X40         85120.0      1
X-LC       XY25        79543.0      3
H-MC       OX40N       70816.0      1
H-LC       X200        67384.0      1
L-MC       XR          57178.0      1
H-SC       MH          48446.0      1
L-LC       XY25        40484.0      1
M-LC       XY25        36748.0      1
           XY24        13776.0      1

In [96]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 41.0 seconds
